In [2]:
using PythonCall

Eof = pyimport("eofs.standard").Eof
np = pyimport("numpy")

Python: <module 'numpy' from '/home/denis/Documents/Uni/Master/MA/visual_analysis/.CondaPkg/env/lib/python3.12/site-packages/numpy/__init__.py'>

In [3]:
include("utils.jl")

normalize_tl_data (generic function with 1 method)

In [17]:
mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"
day_mean_base_path = "/home/denis/workspace/data/ivt_daily_mean"


"/home/denis/workspace/data/ivt_daily_mean"

In [5]:
winter_months = [12, 1, 2, 3]

monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [18]:
daily_tl_data = filter_by_date(build_timeline_data(day_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T12:00:00"), DateTime("2015-01-02T09:00:00"), DateTime("2015-01-03T09:00:00"), DateTime("2015-01-04T09:00:00"), DateTime("2015-01-05T09:00:00"), DateTime("2015-01-06T09:00:00"), DateTime("2015-01-07T09:00:00"), DateTime("2015-01-08T09:00:00"), DateTime("2015-01-09T09:00:00"), DateTime("2015-01-10T09:00:00")  …  DateTime("2100-12-23T09:00:

In [37]:
function pyeof_of_datachunk(datachunk, nmodes; weighted = true, lats = nothing, standard_permute = true, eof_type = :normal)

    if standard_permute
        correct_shape_ds = permutedims(datachunk, (3, 1, 2))
    else
        correct_shape_ds = datachunk
    end


    if weighted
        weights = sqrt.(cos.(deg2rad.(lats)))
        solver = Eof(np.array(correct_shape_ds, dtype=np.float64), weights = np.array(weights, dtype=np.float64))
    else
        solver = Eof(np.array(correct_shape_ds, dtype=np.float64))
    end

    if eof_type == :normal
        eof_res = solver.eofs(neofs = nmodes)
    elseif eof_type == :covariance
        eof_res = solver.eofsAsCovariance(neofs = nmodes)
    elseif eof_type == :correlation 
        eof_res = solver.eofsAsCorrelation(neofs = nmodes)
    else
        ArgumentError("Could not use eof type $(eof_type), use :normal, :correlation or :covariance instead")
    end

    modes_variability = solver.eigenvalues(neigs=nmodes) ./ solver.totalAnomalyVariance() * 100

    EOFResult(pyconvert(Array{Float64, 3}, eof_res), pyconvert(Matrix{Float64}, solver.pcs(npcs=nmodes)), pyconvert(Vector{Float64}, modes_variability))

end

pyeof_of_datachunk (generic function with 1 method)

In [36]:
# create geographical axes
function local_geoaxis_creation!(
    figure::Makie.Figure,
    lonlims::Tuple{<:Number, <:Number}, 
    latlims::Tuple{<:Number, <:Number}; 
    figure_row::Int = 1,
    figure_col::Int = 1,
    title = "",
    dest_projection = "+proj=merc",
    )

    geoaxis = GeoMakie.GeoAxis(
        figure[figure_row, figure_col];
        dest = dest_projection,
        limits=(lonlims, latlims),
        title = title
    )

    return geoaxis
end


function show_eof_modes_of_timeline_pyeofs(
    data::TimelineData,
    nmodes::Int,
    n_seasons::Int,
    filename::String;
    eof_center::Bool = true,
    framerate::Int = 30,
    colormap = :viridis,
    coastline_color = :black,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing,
    fontsize::Int = 12,
    whole_figure_title = nothing,
    eof_type = :normal
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=fontsize) : Figure(size = resolution, fontsize=fontsize)

    if !isnothing(whole_figure_title)
        Label(fig[0, 1:nmodes], whole_figure_title, fontsize= round(1.7 * fontsize))
    end

    all_scopes = get_sliding_time_scopes_by_threshold(data.time, n_seasons)

    current_scope_index = Observable(1)

    axis = Dict()

    dataset_to_eofs = Dict()

    lon_bounds = extrema(data.lons)
    lat_bounds = extrema(data.lats)

    eof_extremas = Tuple{Float64, Float64}[]


    for (i, dataset) in enumerate(data.datasets)
        all_modes_axis = []

        eofs = EOFResult[]

        for scope in all_scopes
            eof_result = pyeof_of_datachunk(dataset.data[:, :, scope], nmodes; lats = data.lats, weighted = true, eof_type = eof_type)
            push!(eof_extremas, extrema(eof_result.spatial_modes))
            push!(eofs, eof_result)
        end

        dataset_to_eofs[dataset.name] = eofs
        
        axis[dataset.name] = all_modes_axis


        for j in 1:nmodes
            title = @lift("$(dataset.name) $(year(data.time[all_scopes[$current_scope_index].start]))-$(year(data.time[all_scopes[$current_scope_index].stop])) Mode $j Variability $(round(dataset_to_eofs[dataset.name][$current_scope_index].modes_variability[j], digits = 2))")
            push!(all_modes_axis, local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=title, figure_row=i, figure_col=j))
        end
    end

    min_val, max_val = reduce((a, b) -> (min(a[1], b[1]), max(a[2], b[2])), eof_extremas)

    Colorbar(fig[length(data.datasets)+1, 1:nmodes], limits = (min_val, max_val), colormap = colormap, vertical = false,  label = "EOF values")
    
    for (i, dataset) in enumerate(data.datasets)
        
        for mode in 1:nmodes
            surface!(axis[dataset.name][mode], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], @lift(dataset_to_eofs[dataset.name][$current_scope_index].spatial_modes[mode, :, :]); shading = shading, colormap = colormap, colorrange = (min_val, max_val))
            lines!(axis[dataset.name][mode], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
        end
    end

    record(fig, filename, eachindex(all_scopes); framerate = framerate) do t
        current_scope_index[] = t
    end

    
    return fig
end

show_eof_modes_of_timeline_pyeofs (generic function with 1 method)

In [38]:
show_eof_modes_of_timeline_pyeofs(monthly_tl_data, 4, 30, "eof_modes_monthly_30seasons_pyeofs_covariance.mp4"; colormap = :broc, framerate = 2, resolution = (2000, 1100), eof_type = :covariance, whole_figure_title = "Leading EOF Modes of Monthly Averages of IVT (30 Winter Period)")